In [ ]:

1 

## Support Vector Machine Practical Procedures to Follow

1. **Getting Data**

2. **Analyzing the Datatypes** (count, type, category)

3. **Visualizing the Data**

4. **Splitting the Dataset into Dependent and Independent Features**

5. **Model Selection**
    - Selecting the model as per the problem statement output
    - Splitting the dataset into training and test data
    - Encoding the text data into numerical values
    - Implementing the model
    - Visualizing the model


In [18]:
import seaborn as sns
import pandas as pd 
df = sns.load_dataset('tips')


In [2]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [10]:
df.value_counts()

total_bill  tip    sex     smoker  day   time    size
13.00       2.00   Female  Yes     Thur  Lunch   2       2
3.07        1.00   Female  Yes     Sat   Dinner  1       1
22.23       5.00   Male    No      Sun   Dinner  2       1
20.69       2.45   Female  No      Sat   Dinner  4       1
            5.00   Male    No      Sun   Dinner  5       1
                                                        ..
15.53       3.00   Male    Yes     Sat   Dinner  2       1
15.69       1.50   Male    Yes     Sun   Dinner  2       1
            3.00   Male    Yes     Sat   Dinner  3       1
15.77       2.23   Female  No      Sat   Dinner  2       1
50.81       10.00  Male    Yes     Sat   Dinner  3       1
Name: count, Length: 243, dtype: int64

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [4]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

#### There is no null values , we'll check the category and how many diffrernt items are there in it and do some EDA 

In [5]:
df['sex'].value_counts()

sex
Male      157
Female     87
Name: count, dtype: int64

In [6]:
df['smoker'].value_counts()

smoker
No     151
Yes     93
Name: count, dtype: int64

In [7]:
df['day'].value_counts()

day
Sat     87
Sun     76
Thur    62
Fri     19
Name: count, dtype: int64

In [8]:
df['time'].value_counts()

time
Dinner    176
Lunch      68
Name: count, dtype: int64

In [9]:
df.columns

Index(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size'], dtype='object')

#### Splitting the dataset into training and testdata before encoding to avoid data leakage 

In [12]:
# spilttin the datat into dependent and independant features 

from sklearn.model_selection import train_test_split

y = df['total_bill']
x = df[['tip', 'sex', 'smoker', 'day', 'time', 'size']]

x_train, x_test , y_train , y_test = train_test_split(x,y,test_size=0.20,random_state=34)

In [13]:
x_train.head()

,tip,sex,smoker,day,time,size
236,1.00,Male,Yes,Sat,Dinner,2
50,2.50,Male,No,Sun,Dinner,2
79,2.71,Male,No,Thur,Lunch,2
128,2.00,Female,No,Thur,Lunch,2
91,3.50,Male,No,Fri,Dinner,2


In [26]:
# encoding the value 

#label encode the binary value and ordinal enocde remaning value 


from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder
from sklearn.compose import ColumnTransformer


ordinal_transformer = OrdinalEncoder()
onehot_transformer = OneHotEncoder()


ordinal_encoded = ['sex', 'smoker', 'time']
onehot_encoded = ['day']

transformer = ColumnTransformer(
    transformers=[
        ('ordinal', ordinal_transformer, ordinal_encoded),
        ('onehot', onehot_transformer, onehot_encoded)
    ],
    remainder='passthrough'  
)


X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=10)


X_train_transformed = transformer.fit_transform(X_train)
X_test_transformed = transformer.transform(X_test)


X_train_transformed = pd.DataFrame(X_train_transformed, columns=transformer.get_feature_names_out())
X_test_transformed = pd.DataFrame(X_test_transformed, columns=transformer.get_feature_names_out())


In [27]:
X_train_transformed.head()

,ordinal__sex,ordinal__smoker,ordinal__time,onehot__day_Fri,onehot__day_Sat,onehot__day_Sun,onehot__day_Thur,remainder__tip,remainder__size
0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,3.48,2.0
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,4.08,2.0
2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,6.00,4.0
3,0.0,1.0,1.0,0.0,0.0,0.0,1.0,3.23,3.0
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,4.71,4.0


In [28]:
# model; training SVM 

from sklearn.svm import SVR

svr = SVR()

svr.fit(X_train_transformed,y_train)

SVR()

In [30]:
y_pred = svr.predict(X_test_transformed)



In [39]:

# accureacy score 

from sklearn.metrics import r2_score,mean_absolute_error

print(r2_score(y_test,y_pred))
print(mean_absolute_error(y_test,y_pred))



0.4466396821573334
4.06989821413618


In [41]:
#Hyper parameter tuninf 
from sklearn.model_selection import GridSearchCV

parameters = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']}

grid = GridSearchCV(SVR(),parameters,refit= True,verbose=3)

grid.fit(X_train_transformed,y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.068 total time=   0.0s
[CV 2/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.011 total time=   0.0s
[CV 3/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.045 total time=   0.0s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.020 total time=   0.0s
[CV 5/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.094 total time=   0.0s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.031 total time=   0.0s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.084 total time=   0.0s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.059 total time=   0.0s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.121 total time=   0.0s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.043 total time=   0.0s
[CV 1/5] END ....C=0.1, gamma=0.01, kernel=rbf;, score=-0.052 total time=   0.0s
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf

GridSearchCV(estimator=SVR(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             verbose=3)

In [42]:
grid.best_params_

{'C': 1000, 'gamma': 0.01, 'kernel': 'rbf'}

In [43]:
y_predV2 = grid.predict(X_test_transformed)

In [44]:
print(r2_score(y_test,y_predV2))
print(mean_absolute_error(y_test,y_predV2))

0.5385222835381605
4.115326940328822


In [50]:
df_results = pd.DataFrame({
    'Actual': y_test,
    'Predicted': y_predV2,
    'Residual': y_test - y_predV2
})

df_results.head()

,Actual,Predicted,Residual
162,16.21,14.384068,1.825932
60,20.29,19.953681,0.336319
61,13.81,16.281421,-2.471421
63,18.29,30.290451,-12.000451
69,15.01,16.586169,-1.576169
